In [28]:
# importing libraries

import pandas as pd
import numpy as np
import os
import sqlite3
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [29]:
# setting environments

url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
db_name = 'Banks.db'
table_name = 'Largest_banks'
table_attribs = ['Name', 'MC_USD_Billion']
table_attribs_1 = ['Name', 'MC_USD_Billion', 'MC_GBP_Billion', 'MC_EUR_Billion', 'MC_INR_Billion']
csv_path = './Largest_banks_data.csv'

sql_connection = sqlite3.connect(db_name)

In [30]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('tbody')
    headers = table[0].find_all('th')
    rows = table[0].find_all('tr')
    records = []
    for i in rows[1:]:
        data = i.find_all('td')
        row = [x.text.strip() for x in data[1:]]
        records.append(row)
        df = pd.DataFrame(records, columns = table_attribs)

    return df

#extract(url, table_attribs)
df = extract(url, table_attribs)

In [31]:
def transform(df):
    ''' This function converts the market cap column to float and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies according to exchange rate'''
    
    exchange_rate = pd.read_csv(r"Downloads\exchange_rate.csv")
    exchange_rate = exchange_rate.set_index('Currency').to_dict()['Rate']
    df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(float)
    df['MC_GBP_Billion'] = [np.round(x*0.82,2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*0.93,2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*82.95,2) for x in df['MC_USD_Billion']]

    return df

df = transform(df)

In [32]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

    df.to_csv(csv_path)


In [33]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    
    df.to_sql(table_name, sql_connection, if_exists = 'replace', index = False)


In [34]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

    ''' Here, you define the required entities and call the relevant
    functions in the correct order to complete the project. Note that this
    portion is not inside any function.'''

In [35]:
# logging process

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing''' 
    
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')
        
        
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('World_Economies.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

query_statement = f'SELECT * FROM {table_name}'
query_statement1 = f'SELECT AVG(MC_GBP_Billion) FROM {table_name}'
query_statement2 = f'SELECT Name from {table_name} LIMIT 5'
run_query(query_statement, sql_connection)
run_query(query_statement1, sql_connection)
run_query(query_statement2, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

log_progress('Server Connection closed')

SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          354.99   
1                          Bank of America          231.52          189.85   
2  Industrial and Commercial Bank of China          194.56          159.54   
3               Agricultural Bank of China          160.68          131.76   
4                                HDFC Bank          157.91          129.49   
5                              Wells Fargo          155.87          127.81   
6                        HSBC Holdings PLC          148.90          122.10   
7                           Morgan Stanley          140.83          115.48   
8                  China Construction Bank          139.82          114.65   
9                            Bank of China          136.81          112.18   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    